In [1]:
import yahoo_fin
import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import string
import urllib.request 
from urllib.request import urlopen
import http.cookiejar
import yahoo_fin.stock_info as si
import re
import json
import csv
from io import StringIO
import numpy as np
import yfinance as yf
import yahoo_fin
import yahoo_fin.stock_info as si
from urllib.request import urlopen
from bs4 import BeautifulSoup

## Inputting Parameters
<br>

In [2]:
def get_roe(ticker):
    url = "https://query1.finance.yahoo.com/v10/finance/quoteSummary/" + ticker + "?modules=financialData"
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")
    
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    
    # get text
    text = soup.get_text()
    
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    
    index_1 = text.find("returnOnEquity")
    index_2 = text.find("fmt", index_1) + 6
    index_3 = text.find("}", index_2) - 2
    x = float(text[index_2 : index_3])/100
    roe = round(x, 5)
    
    return roe


In [3]:
def get_dividend(ticker):    
    ticker = "amzn"
    quote_table = si.get_quote_table(ticker)
    str = quote_table["Forward Dividend & Yield"]
    if str == "N/A (N/A)":
        return 0 
    else:
        index = str.find("(")
        dvdt = float(str[0 : index - 1])
        return dvdt

In [4]:
def get_SH_equity(ticker):
    balance_sheet = si.get_financials(ticker, yearly = True, quarterly = False)['yearly_balance_sheet']
    x = balance_sheet.loc['totalStockholderEquity'][0].item()
    return x

In [5]:
def  get_shares_out(ticker):
    t = yf.Ticker(ticker)
    x = t.info
    shares_out = x['sharesOutstanding']
    return shares_out

In [6]:
def get_gr(ticker):
    t = yf.Ticker(ticker)
    x = t.info
    k = x['earningsGrowth']
    return k

In [7]:
ticker = input("Enter a ticker: ")

roe = get_roe(ticker)
dvd = get_dividend(ticker) 
stk_eq = get_SH_equity(ticker) #Stock-holder equity
shares_out = get_shares_out(ticker)
gr = get_gr(ticker)
print("The ROE is", roe)
print("The Dividend is", dvd)
print("The Shareholder's Equity is", stk_eq)
print("The Outstanding Shares are", shares_out)
print("The Growth Rate is", gr)

Enter a ticker: aapl
The ROE is 1.7546
The Dividend is 0
The Shareholder's Equity is 50672000000
The Outstanding Shares are 15908100096
The Growth Rate is 0.048


In [8]:
stk_eq_PER = stk_eq / shares_out #Shareholders equity per share
stk_eq_PER_y10 = stk_eq_PER * (1 + gr)**10 #Shareholders equity per share grown till time 10 
divdnt_y10 = [dvd*(1 + gr)**i*(1 + 0.09)**(-i) for i in range(1,11)] #Growing dividends every year by the growth rate, and then taking npv
y10_income = stk_eq_PER_y10 * roe # Income generated per share in Year 10 using shareholders equity
income = y10_income / 0.09 #Assume perpetuity of income generated per share using shareholders_equity. Discount rate used is 0.09

income_npv = income/(1 + 0.09)**10 #income NPV today 
sum_divdnt = np.sum(np.array(divdnt_y10)) #Sum of income NPV and sum of dividents
final_value = sum_divdnt + income_npv
print("The Intrinsic value of $", ticker, " \nby ROE valuation method is", final_value)

The Intrinsic value of $ aapl  
by ROE valuation method is 41.92115056132375
